In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Bob Dylan,"Going, Going, Gone",I've just reached a place \r\nWhere the willo...
1,Madonna,Iconic,[Intro: Mike Tyson] \r\nI'm the best the worl...
2,New Order,Everything's Gone Green,"Help me, somebody help me \r\nI wonder where ..."
3,Bob Dylan,From A Buick 6,"I got this graveyard woman, you know she keeps..."
4,Frank Zappa,Alley Cat,Captain beefheart (lead vocals) \r\nFrank zap...
5,Queen Adreena,Are The Songs My Disease?,(K. J. Garside C. Gray) \r\nI used to know th...
6,Matt Redman,Take The World But Give Me Jesus,I've thrown it all away that I might gain a li...
7,Bette Midler,The Last Time,"Let this be the last time, let this be the las..."
8,Christmas Songs,Old Toy Trains,"Old toy trains, little toy tracks \r\nLittle ..."
9,Rod Stewart,I've Got A Crush On You,How glad the many millions \r\nOf Toms and Di...


In [9]:
df['text'][0]

'I\'ve just reached a place  \r\nWhere the willow don\'t bend.  \r\nThere\'s not much more to be said  \r\nIt\'s the top of the end.  \r\nI\'m going,  \r\nI\'m going,  \r\nI\'m gone.  \r\n  \r\nI\'m closin\' the book  \r\nOn the pages and the text  \r\nAnd I don\'t really care  \r\nWhat happens next.  \r\nI\'m just going,  \r\nI\'m going,  \r\nI\'m gone.  \r\n  \r\nI been hangin\' on threads,  \r\nI been playin\' it straight,  \r\nNow, I\'ve just got to cut loose  \r\nBefore it gets late.  \r\nSo I\'m going,  \r\nI\'m going,  \r\nI\'m gone.  \r\n  \r\nGrandma said, "Boy, go and follow your heart  \r\nAnd you\'ll be fine at the end of the line.  \r\nAll that\'s gold isn\'t meant to shine.  \r\nDon\'t you and your one true love ever part."  \r\n  \r\nI been walkin\' the road,  \r\nI been livin\' on the edge,  \r\nNow, I\'ve just got to go  \r\nBefore I get to the ledge.  \r\nSo I\'m going,  \r\nI\'m just going,  \r\nI\'m gone.\r\n\r\n'

In [10]:
df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [14]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [17]:
similarity[0]

array([1.        , 0.01452684, 0.06435059, ..., 0.02176712, 0.0666039 ,
       0.03948305])

In [27]:
df[df['song'] == 'Going, Going, Gone'].index[0]

0

In [24]:
def recommendation(song_df):
    # Check if the song exists in the dataframe
    if song_df not in df['song'].values:
        return f"Song '{song_df}' not found in the dataset."
    
    # Get the index of the song in the DataFrame
    idx = df[df['song'] == song_df].index[0]
    
    # Calculate distances and sort them in descending order
    distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])
    
    # Collect the recommended songs
    songs = []
    for m_id in distances[1:21]:  # Exclude the first one because it's the song itself
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [29]:
recommendation("I've Got A Crush On You")

['Why',
 'All Of Me',
 'Dream The Night Away',
 'The Night Inside Me',
 'Quite Clear',
 "I Don't Know Why",
 'Questions',
 'Last Train To London',
 'Eyes Of Innocence',
 'Losing Grip',
 'Human Nature',
 'You Opened My Eyes',
 'The Accusation Chair',
 'When I Look Into Your Eyes',
 "Why Don't We",
 "I Don't Know",
 'Look At Me Now',
 'Why Not Me',
 'Better Left Forgotten',
 'Missing']

In [30]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))